In [1]:
import attr
import xsimlab as xs


In [2]:
def create_process(dim):
    
    @xs.process
    class A:
        x = xs.index(dims=dim)
        
        def initialize(self):
            self.x = [0, 1, 2, 3]
    
    return A

In [3]:
B = create_process('y')

In [4]:
B

__main__.create_process.<locals>.A

In [5]:
C = create_process('z')

In [6]:
C

__main__.create_process.<locals>.A

In [7]:
xs.process_info(B)

<A  (xsimlab process)>
Variables:
    x          [out] ('y',) 
Simulation stages:
    initialize


In [8]:
xs.process_info(C)

<A  (xsimlab process)>
Variables:
    x          [out] ('z',) 
Simulation stages:
    initialize


In [2]:
@xs.process
class X:
    pass

In [11]:
# do not decorate A with attr.s or xs.process!!

class A(X):
    index = xs.index(dims=('not_initialized'))
    label = xs.variable(intent='in')
    dim = xs.variable(intent='in')
    
    output = xs.variable(intent='out')

    def initialize(self):
        self.index = [f"{self.label}-{i}" for i in range(self.dim)]
        
        print(self.index)

In [12]:
def make_process(cls_name, label):
    new_cls = type(cls_name, A.__bases__, dict(A.__dict__))
    
    new_cls.index.metadata['dims'] = label
    
    new_cls.output.metadata['dims'] = ((label, 'time'),)
    
    
    return xs.process(new_cls)


In [13]:
B = make_process('B', 'y')

In [14]:
B

__main__.B

In [15]:
xs.process_info(B)

<B  (xsimlab process)>
Variables:
    index      [out] y 
    label       [in]
    dim         [in]
    output     [out] ('y', 'time') 
Simulation stages:
    initialize


In [26]:
model = xs.Model({'Bx':B})

In [27]:
in_ds = xs.create_setup(model=model,
                clocks={'clock': [0, 1]},
               input_vars={
                   'Bx__dim':20,
                   'Bx__label':'Phytoplankton'
               },
                output_vars={}
               )

In [19]:
with model:
    out_ds = in_ds.xsimlab.run()

['Phytoplankton-0', 'Phytoplankton-1', 'Phytoplankton-2', 'Phytoplankton-3', 'Phytoplankton-4', 'Phytoplankton-5', 'Phytoplankton-6', 'Phytoplankton-7', 'Phytoplankton-8', 'Phytoplankton-9', 'Phytoplankton-10', 'Phytoplankton-11', 'Phytoplankton-12', 'Phytoplankton-13', 'Phytoplankton-14', 'Phytoplankton-15', 'Phytoplankton-16', 'Phytoplankton-17', 'Phytoplankton-18', 'Phytoplankton-19']


In [20]:
out_ds

<xarray.Dataset>
Dimensions:    (clock: 2, y: 20)
Coordinates:
  * clock      (clock) int64 0 1
Dimensions without coordinates: y
Data variables:
    Bx__dim    int64 20
    Bx__label  <U13 'Phytoplankton'
    index      (y) <U16 'Phytoplankton-0' ... 'Phytoplankton-19'

In [64]:
#C = make_process('C', 'z')

In [100]:
new_cls = type('Hey', A.__bases__, dict(A.__dict__))

In [102]:
new_cls.index.metadata['dims'] = (('q', 'P'),)

new_cls.index.metadata['dims']

(('q', 'P'),)

In [103]:
new_cls.dim.metadata

{'var_type': <VarType.VARIABLE: 'variable'>,
 'dims': ((),),
 'intent': <VarIntent.OUT: 'out'>,
 'groups': (),
 'static': False,
 'attrs': {},
 'description': '',
 'encoding': {},
 'default': 3}

In [67]:

new_cls.q.metadata['default']

KeyError: 'default'

In [32]:
xs.process_info(C)

<C  (xsimlab process)>
Variables:
    x          [out] z 
    q          [out] z or time 
Simulation stages:
    initialize


In [29]:
from xsimlab import RuntimeHook, runtime_hook

import time

In [30]:
class PrintStepTime(RuntimeHook):

    @runtime_hook('run_step', 'model', 'pre')
    def start_step(self, model, context, state):
        self._start_time = time.time()
        print(f"Starting step {context['step']}")

    @runtime_hook('run_step', 'model', 'post')
    def finish_step(self, model, context, state):
        step_time = time.time() - self._start_time
        print(f"Step {context['step']} took {step_time} seconds")


In [31]:
with PrintStepTime():
    in_ds.xsimlab.run(model=model)

['Phytoplankton-0', 'Phytoplankton-1', 'Phytoplankton-2', 'Phytoplankton-3', 'Phytoplankton-4', 'Phytoplankton-5', 'Phytoplankton-6', 'Phytoplankton-7', 'Phytoplankton-8', 'Phytoplankton-9', 'Phytoplankton-10', 'Phytoplankton-11', 'Phytoplankton-12', 'Phytoplankton-13', 'Phytoplankton-14', 'Phytoplankton-15', 'Phytoplankton-16', 'Phytoplankton-17', 'Phytoplankton-18', 'Phytoplankton-19']
Starting step 0
Step 0 took 4.100799560546875e-05 seconds
